![](https://github.com/rajeevratan84/ModernComputerVision/raw/main/logo_MCV_W.png)

# **Neural Style Transfer with OpenCV**

####**In this lesson we'll learn how to use pre-trained Models to implement Neural Style Transfer in OpenCV**

![](https://github.com/rajeevratan84/ModernComputerVision/raw/main/NSTdemo.png)

**About Neural Style Transfers**

Introduced by Leon Gatys et al. in 2015, in their paper titled “[A Neural Algorithm for Artistic Style](https://arxiv.org/abs/1508.06576)”, the Neural Style Transfer algorithm went viral resulting in an explosion of further work and mobile apps.

Neural Style Transfer enables the artistic style of an image to be applied to another image! It copies the color patterns, combinations, and brush strokes of the original source image and applies it to your input image. And is one the most impressive implementations of Neural Networks in my opinion.

![](https://github.com/rajeevratan84/ModernComputerVision/raw/main/NST.png)

In [1]:
!wget https://moderncomputervision.s3.eu-west-2.amazonaws.com/NeuralStyleTransfer.zip
!wget https://github.com/rajeevratan84/ModernComputerVision/raw/main/city.jpg
!unzip -qq NeuralStyleTransfer.zip

--2023-06-02 09:25:17--  https://moderncomputervision.s3.eu-west-2.amazonaws.com/NeuralStyleTransfer.zip
Resolving moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)... 52.95.148.158
Connecting to moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)|52.95.148.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 186982232 (178M) [application/zip]
Saving to: ‘NeuralStyleTransfer.zip’

NeuralStyleTransfer 100%[===================>] 178.32M  21.1MB/s    in 9.3s    

2023-06-02 09:25:27 (19.3 MB/s) - ‘NeuralStyleTransfer.zip’ saved [186982232/186982232]

--2023-06-02 09:25:27--  https://github.com/rajeevratan84/ModernComputerVision/raw/main/city.jpg
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rajeevratan84/Mo

In [2]:
import numpy as np
import time
import cv2
import os
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt 
from IPython.display import HTML
from base64 import b64encode

def imshow(title = "Image", image = None, size = 10):
  h, w = image.shape[:2]
  aspect_ratio = w/h
  plt.figure(figsize=(size * aspect_ratio,size))
  plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title(title)
  plt.show()


### **Implement Neural Style Transfer using pretrained Models**

We use pretrained t7 PyTorch models that can be imported using ``cv2.dnn.readNetFromTouch()```

These models we're using come from the paper *Perceptual Losses for Real-Time Style Transfer and Super-Resolution* by Johnson et al. 

They improved proposing a Neural Style Transfer algorithm that performed 3 times faster by using a super-resolution-like problem based on perceptual loss function.

In [3]:
model_file_path = "NeuralStyleTransfer/models/"
model_file_paths = [f for f in listdir(model_file_path) if isfile(join(model_file_path, f))]

img = cv2.imread("city.jpg")
mean = np.mean(img, axis=(0,1))

# Loop through and applying each model style our input image
for (i,model) in enumerate(model_file_paths):

  print(str(i+1) + ". Using Model: " + str(model)[:-3])    
  style = cv2.imread("NeuralStyleTransfer/art/"+str(model)[:-3]+".jpg")
  # loading our neural style transfer model 
  neuralStyleModel = cv2.dnn.readNetFromTorch(model_file_path + model)

  # Let's resize to a fixed height of 640 (free to change)
  height, width = int(img.shape[0]), int(img.shape[1])
  newWidth = int((640 / height) * width)
  resizedImg = cv2.resize(img, (newWidth, 640), interpolation=cv2.INTER_AREA)

  # Create our blob from the image and then perform a forward pass run of the network
  inpBlob = cv2.dnn.blobFromImage(resizedImg, 1.0, (newWidth, 640), (mean[2], mean[1] ,mean[0]), swapRB=False, crop=False)

  neuralStyleModel.setInput(inpBlob)
  output = neuralStyleModel.forward()

  # Reshaping the output tensor, adding back  the mean subtraction and re-ordering the channels 
  output = output.reshape(3, output.shape[2], output.shape[3])
  output[0] += 103.939
  output[1] += 116.779
  output[2] += 123.68
  output /= 255
  output = output.transpose(1, 2, 0)
    
  #Display our original image, the style being applied and the final Neural Style Transfer
  imshow("Original", img)
  imshow("Style", style)
  imshow("Neural Style Transfers", output)

Output hidden; open in https://colab.research.google.com to view.

## **Using the ECCV16 Updated NST Algorithm**

In Ulyanov et al.’s 2017 publication, *Instance Normalization: The Missing Ingredient for Fast Stylization*, it was found that swapping batch normalization for instance normalization (and applying instance normalization at both training and testing), leads to even faster real-time performance and arguably more aesthetically pleasing results as well.

Let's now use the models used by Johnson et al. in their ECCV paper.



In [4]:
model_file_path = "NeuralStyleTransfer/models/ECCV16/"
model_file_paths = [f for f in listdir(model_file_path) if isfile(join(model_file_path, f))]

img = cv2.imread("city.jpg")
mean = np.mean(img, axis=(0,1))

for (i,model) in enumerate(model_file_paths):

  print(str(i+1) + ". Using Model: " + str(model)[:-3])    
  style = cv2.imread("NeuralStyleTransfer/art/"+str(model)[:-3]+".jpg")

  neuralStyleModel = cv2.dnn.readNetFromTorch(model_file_path+model)

  height, width = int(img.shape[0]), int(img.shape[1])
  newWidth = int((640 / height) * width)
  resizedImg = cv2.resize(img, (newWidth, 640), interpolation = cv2.INTER_AREA)

  inpBlob = cv2.dnn.blobFromImage(resizedImg, 1.0, (newWidth, 640), (mean[2], mean[1] ,mean[0]), swapRB=False, crop=False)

  neuralStyleModel.setInput(inpBlob)
  output = neuralStyleModel.forward()

  output = output.reshape(3, output.shape[2], output.shape[3])
  output[0] += 103.939
  output[1] += 116.779
  output[2] += 123.68
  output /= 255
  output = output.transpose(1, 2, 0)
    
  imshow("Original", img)
  imshow("Style", style)
  imshow("Neural Style Transfers", output)

Output hidden; open in https://colab.research.google.com to view.

In [5]:
!wget https://github.com/rajeevratan84/ModernComputerVision/raw/main/dj.mp4

--2023-06-02 09:26:09--  https://github.com/rajeevratan84/ModernComputerVision/raw/main/dj.mp4
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rajeevratan84/ModernComputerVision/main/dj.mp4 [following]
--2023-06-02 09:26:09--  https://raw.githubusercontent.com/rajeevratan84/ModernComputerVision/main/dj.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174741 (171K) [application/octet-stream]
Saving to: ‘dj.mp4’

dj.mp4              100%[===================>] 170.65K  --.-KB/s    in 0.01s   

2023-06-02 09:26:10 (11.3 MB/s) - ‘dj.mp4’ saved [174741/174741]



In [6]:
model_file_path = "NeuralStyleTransfer/models/ECCV16/starry_night.t7"

cap = cv2.VideoCapture('dj.mp4')

w = int(cap.get(3)) 
h = int(cap.get(4))

out = cv2.VideoWriter('NST_Starry_Night.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (w, h))

style = cv2.imread("NeuralStyleTransfer/art/starry_night.jpg")
i = 0
mean = np.mean(img, axis=(0,1))

while True:

  ret, img = cap.read()
  
  if ret == True:

    i += 1
    print("Completed {} Frame(s)".format(i))
    neuralStyleModel = cv2.dnn.readNetFromTorch(model_file_path)

    height, width = int(img.shape[0]), int(img.shape[1])
    newWidth = int((640 / height) * width)
    resizedImg = cv2.resize(img, (newWidth, 640), interpolation = cv2.INTER_AREA)

    inpBlob = cv2.dnn.blobFromImage(resizedImg, 1.0, (newWidth, 640), (mean[2], mean[1] ,mean[0]), swapRB=False, crop=False)

    neuralStyleModel.setInput(inpBlob)
    output = neuralStyleModel.forward()

    output = output.reshape(3, output.shape[2], output.shape[3])
    output[0] += 103.939
    output[1] += 116.779
    output[2] += 123.68
    output /= 255
    output = output.transpose(1, 2, 0)
      
    vid_output = (output * 255).astype(np.uint8)
    vid_output = cv2.resize(vid_output, (w, h), interpolation = cv2.INTER_AREA)
    out.write(vid_output)
  else:
    break

cap.release()
out.release()

Completed 1 Frame(s)
Completed 2 Frame(s)
Completed 3 Frame(s)
Completed 4 Frame(s)
Completed 5 Frame(s)
Completed 6 Frame(s)
Completed 7 Frame(s)
Completed 8 Frame(s)
Completed 9 Frame(s)
Completed 10 Frame(s)
Completed 11 Frame(s)
Completed 12 Frame(s)
Completed 13 Frame(s)
Completed 14 Frame(s)
Completed 15 Frame(s)
Completed 16 Frame(s)
Completed 17 Frame(s)
Completed 18 Frame(s)
Completed 19 Frame(s)
Completed 20 Frame(s)
Completed 21 Frame(s)
Completed 22 Frame(s)
Completed 23 Frame(s)
Completed 24 Frame(s)
Completed 25 Frame(s)
Completed 26 Frame(s)
Completed 27 Frame(s)
Completed 28 Frame(s)
Completed 29 Frame(s)
Completed 30 Frame(s)
Completed 31 Frame(s)
Completed 32 Frame(s)
Completed 33 Frame(s)


## **Display your video**

In [7]:
!ffmpeg -i /content/NST_Starry_Night.avi NST_Starry_Night.mp4 -y

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [8]:
video_path = '/content/NST_Starry_Night.mp4'
 
mp4 = open(video_path, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=600 controls><source src="{data_url}" type="video/mp4">
</video>""")

Output hidden; open in https://colab.research.google.com to view.

## **Want to train your own NST Model?**

## **Look at later sections of the course where we take a look at Implementing our very own Deep Learning NST Algorithm**

Alternatively, give this github repo a shot and try it yourself - https://github.com/jcjohnson/fast-neural-style